# Random Number Generators: LCG, Shuffle, and Lagged Fibonacci (Julia)

A complete conceptual + computational walk-through

Random Number Generators (RNGs) are the foundation of Monte Carlo simulations, numerical integration, stochastic sampling, molecular dynamics, and probabilistic machine-learning workflows.

In this notebook, we implement and study:
	1.	Linear Congruential Generator (LCG)
	2.	Shuffle RNG (Bays–Durham, improves LCG)
	3.	Lagged Fibonacci Generator (LFG)
	4.	Autocorrelation tests
	5.	Moment tests
	6.	Monte Carlo importance sampling integrals

Each method is explained with math, Julia code, and analysis.

⸻

## 1. Linear Congruential Generator (LCG)

### 1.1 What is an LCG?

An LCG is one of the oldest and simplest pseudorandom number generators.
It generates integers recursively via:

$$
X_{n+1} = (a X_n + c) \bmod m,
$$

and outputs a uniform number in $(0,1)$ by:

$$
r_n = \frac{X_n}{m}.
$$

We choose the common form with $c = 0$ (multiplicative LCG):

$$
X_{n+1} = (aX_n) \bmod m.
$$

### 1.2 Why LCGs matter
	•	Extremely fast
	•	Portable
	•	Easy to implement
	•	Used historically in many scientific libraries

But they suffer from structural correlations:
	•	Points lie on hyperplanes
	•	Poor higher-dimensional performance
	•	Bad low-bit randomness
	•	Strong serial correlations

This motivates improved generators (Shuffle, Lagged Fibonacci).

In [7]:
using Statistics
using Plots

RNG of the form:

X_{n+1} = (a X_n) \mod m

The output random number is:

r_{n+1} = \frac{X_{n+1}}{m}

LCG is very fast, but suffers from correlations and lattice structure in high dimensions.

We use:
	•	m = 2^{31} - 1
	•	a = 17869

In [1]:
mutable struct LCG
    state::Int64
    a::Int64
    m::Int64
end

# Constructor
LCG(seed::Int=123456789; a::Int=17869, m::Int=Int(2^31 - 1)) =
    LCG(mod(seed, m), a, m)

# Generate a single random number
function rand_lcg!(rng::LCG)
    rng.state = (rng.state * rng.a) % rng.m
    return rng.state / rng.m
end

# Generate a sequence
function lcg_sequence(n::Int; seed::Int=1234)
    rng = LCG(seed)
    out = Vector{Float64}(undef, n)
    @inbounds for i in 1:n
        out[i] = rand_lcg!(rng)
    end
    return out
end

lcg_sequence (generic function with 1 method)

In [2]:
lcg_vals = lcg_sequence(10_000)
println("First 5 LCG values: ", lcg_vals[1:5])

First 5 LCG values: [0.010267992508722465, 0.47875813836173997, 0.9291743859318897, 0.41710221693716115, 0.19951445013262073]


## 1.4 Autocorrelation Test for RNG Quality

To test serial correlation, we compute:

$$
C_k = \frac{1}{N} \sum_{i=1}^{N-k}
(r_i - 0.5)(r_{i+k} - 0.5).
$$

If numbers are independent, $C_k \approx 0$.

In [3]:
function autocorrelations(r::Vector{Float64}, maxlag::Int=3)
    N = length(r)
    C = zeros(maxlag)
    @inbounds for k in 1:maxlag
        s = 0.0
        for i in 1:(N-k)
            s += (r[i] - 0.5) * (r[i+k] - 0.5)
        end
        C[k] = s / N
    end
    return C
end

println("LCG autocorrelations: ", autocorrelations(lcg_vals))

LCG autocorrelations: [0.0018383911365287126, 0.00033257697875941256, 0.0006767718141938466]


# 2. Shuffle RNG (Bays–Durham Shuffle)

## 2.1 Why do we need a Shuffle RNG?

LCGs are fast but have serial correlations, i.e.:
	•	$r_i$ depends strongly on $r_{i-1}$
	•	Values tend to fall on structure/hyperplanes

The Shuffle RNG improves LCG quality by:
	•	Keeping a table of past values
	•	Picking from the table using a random index
	•	Replacing entries with fresh LCG values

This “scrambles” correlations and produces a far better sequence.

## 2.2 Math behind the Shuffle RNG

Algorithm:
	1.	Fill a table:
$$
T_j = r_j, \quad j = 1,\ldots,L
$$
	2.	Maintain a value $y$ from the previous step
	3.	Compute:
$$
j = \lfloor Ly \rfloor + 1
$$
	4.	Output $T_j$, replace it with a new LCG number.

This has much lower correlation.

In [4]:
function shuffle_rng(n::Int; table_len::Int=100, seed::Int=1234)
    rng = LCG(seed)
    # Step 1: Initialize table
    table = [rand_lcg!(rng) for _ in 1:table_len]

    # Step 2: Starting value for index selection
    y = rand_lcg!(rng)
    out = Vector{Float64}(undef, n)

    @inbounds for i in 1:n
        j = clamp(floor(Int, y * table_len) + 1, 1, table_len)
        out[i] = table[j]
        y = table[j]               # feedback
        table[j] = rand_lcg!(rng)  # replace
    end
    return out
end

shuf_vals = shuffle_rng(10_000)
println("Shuffle sample: ", shuf_vals[1:5])
println("Shuffle autocorrelations: ", autocorrelations(shuf_vals))

Shuffle sample: [0.807263942345634, 0.9791460414319979, 0.9575325371499791, 0.2913307069294763, 0.15336508357588438]
Shuffle autocorrelations: [0.0004396398339355917, 0.0018429612799164556, 0.00010862308281634805]


# 3. Lagged Fibonacci Generator (LFG)

A much higher-quality generator than LCG.

⸻

## 3.1 Math Behind LFG

Uses lagged recurrence:

$$
X_n = (X_{n-j} , \oplus , X_{n-k}) \bmod m
$$

where $\oplus$ can be:
	•	addition
	•	subtraction
	•	multiplication
	•	XOR

We use the multiplicative version:

$$
X_n = (X_{n-24}X_{n-55}) \bmod m.
$$

This generator:
	•	Has very long periods
	•	Has low correlation
	•	Works well for Monte Carlo


In [5]:
function lfg_sequence(n::Int; j::Int=24, k::Int=55, seed::Int=1234)
    m = Int(2^31 - 1)
    a = 17869
    rng = LCG(seed; a=a, m=m)

    # Buffer seeded with LCG integers
    buf = Vector{Int}(undef, k)
    for i in 1:k
        buf[i] = Int(floor(rand_lcg!(rng) * m))
    end

    out = Vector{Float64}(undef, n)
    pos = 1

    @inbounds for t in 1:n
        idx_j = (pos - j - 1) % k + 1
        idx_k = (pos - k - 1) % k + 1

        newval = (buf[idx_j] * buf[idx_k]) % m
        buf[pos] = newval
        out[t] = newval / m

        pos = (pos % k) + 1
    end

    return out
end

lfg_vals = lfg_sequence(10_000)
println("LFG autocorrelations: ", autocorrelations(lfg_vals))

LFG autocorrelations: [0.243260183454601, 0.23896540426619303, 0.2751604251899047]


In [ ]:
# 4. Moment Tests

Uniform distribution satisfies:

$$
\mathbb{E}[r^n] = \frac{1}{n+1}, \quad r \sim U[0,1].
$$

For $n = 1,2,3$:

In [8]:
function raw_moments_with_error(r::Vector{Float64}; pows=(1,2,3))
    N = length(r)
    moments = zeros(Float64, length(pows))
    errors = zeros(Float64, length(pows))

    for (ix, n) in enumerate(pows)
        vals = r .^ n
        moments[ix] = mean(vals)
        target = 1/(n+1)
        errors[ix] = sqrt(mean((vals .- target).^2))
    end
    return moments, errors
end

println("LCG moments & errors: ", raw_moments_with_error(lcg_vals))
println("Shuffle moments & errors: ", raw_moments_with_error(shuf_vals))
println("LFG moments & errors: ", raw_moments_with_error(lfg_vals))

LCG moments & errors: ([0.5032101913365583, 0.33618640583031123, 0.25238437469420494], [0.2880559225111557, 0.2982757505668904, 0.28375959451188704])
Shuffle moments & errors: ([0.5032609017514907, 0.33620974104785856, 0.2523992830593886], [0.28800840143365236, 0.2982895576775185, 0.2838649444749259])
LFG moments & errors: ([-0.022260655253455344, 0.2204540096577337, -0.023987110235578042], [0.7019363681354521, 0.318931828880712, 0.42058308659973676])
